In [1]:
import os
import dotenv
import openai
import evaluation
dotenv.load_dotenv()
openai.api_key = os.getenv("GPT_SECRET_KEY")

In [16]:
def _read_file(path: str) -> str:
    with open(path, encoding="utf-8") as f:
        return f.read()

response = openai.Completion.create(
  model="gpt-3.5-turbo-instruct",
  prompt=_read_file("nagini_examples/cot_partial_prompt.txt"),
  max_tokens=500,
  suffix="-------------------"
)

In [17]:
print(response)
print(response["choices"][0]["text"])

{
  "id": "cmpl-8IOYIPTtkFXzAxFMuGthSUw2xtHm0",
  "object": "text_completion",
  "created": 1699394974,
  "model": "gpt-3.5-turbo-instruct",
  "choices": [
    {
      "text": "# In order to address this new verification error, we need to add an Unfold statement at line 15\n# This will allow us to access the is_list(ptr) predicate in every iteration of the loop\n# We also need to add an Unfold statement at line 19 to access is_list(prev) when assigning the value of ptr to prev\n# Additionally, we need to add a Fold statement after line 19 to ensure we are still verifying the is_list(prev) predicate after assignment\n# Then, we add a Fold statement at line 21 to ensure that we are still verifying the is_list(ptr) predicate after assignment\n# Finally, we need to add a Fold statement after line 23 to ensure that we are verifying the is_list(ptr) predicate before returning\ndef reverseList(head: Node) -> Optional[Node]:\n    \"\"\"Reverses the list and returns the new head.\"\"\"\n    Req

In [12]:
evaluation = evaluation.Evaluation("config_cot.json")
program = """
def append(head: Node, val: int) -> None:
    Requires(is_list(head))
    Ensures(is_list(head))
    if head.next is None:
        Unfold(is_list(head))
        n = Node(val)
        Fold(is_list(n))
        head.next = n
        Fold(is_list(head))
    else:
        Unfold(is_list(head))
        append(head.next, val)
        Fold(is_list(head))
"""
evaluation._verify_program_snippet("list", program)

response ['', 'Verification failed', 'Errors:', 'Conditional statement might fail. There might be insufficient permission to access head.next. (tmp.py@25.7)', 'Verification took 2.61 seconds.']


VerificationResult(status='Verification failed', errors='Conditional statement might fail. There might be insufficient permission to access head.next.', line_no='5.7')

In [13]:
print(_)

Verification failed: Conditional statement might fail. There might be insufficient permission to access head.next. at line 5.7


In [1]:
import model
import data
dataset = data.Data("config_cot.json")
completion = model.Completion("gpt-3.5-turbo-instruct", dataset)
prompt = completion.get_prompt("append", with_errors=True)
print(prompt)

<task-description>
You are an assistant that given a python program, 
annotates it with appropriate Nagini annotations so that verification succeeds.

Nagini is a static verifier for Python. 
Our aim is to given a statically typed Python program, 
to come up with appropriate preconditions (e.g. Requires(is_list(head))), postcondition (e.g. Ensures(is_list(Result()))),
loop invariants (Invariant(<assertion>)), predicate fold/unfolds (e.g. Fold(is_list(head)) / Unfold(is_list(head))) 
so that the program verifies correctly.
The user will provide Python code and the verification errors. 
You must add or change the specifications so that the resulting code verifies correctly. 
First, think step-by-step: explain your reasoning as python code comments e.g.
# Based on the verification error there might be insufficient permissions ...
# Hence, I should add an Unfold at line 7 ...
Following the comments, return only the code without any explanation or wrapping. e.g.
def append(head: Node, val: 

In [1]:
import evaluation
evaluation = evaluation.Evaluation("config_cot.json", "gpt-3.5-turbo")
evaluation.run_example("append", k=3,n=2, with_errors=True)

Running example: append ; attempt: 1 ; error depth: 1
Generated program from GPT:
def append(head: Node, val: int) -> None:
    """Appends a new node with the given value to the end of the list."""
    Requires(is_list(head))
    Ensures(is_list(head))
    Unfold(is_list(head))
    if head.next is None:
        n = Node(val)
        Fold(is_list(n))
        head.next = n
    else:
        Unfold(is_list(head))
        append(head.next, val)
        Fold(is_list(head))
    Fold(is_list(head))
response ['', 'Verification failed', 'Errors:', 'Unfold might fail. There might be insufficient permission to access is_list(head). (tmp.py@31.8)', 'Verification took 5.85 seconds.']
Verification result:
 Verification failed: Unfold might fail. There might be insufficient permission to access is_list(head). at line 11.8 


Running example: append ; attempt: 1 ; error depth: 2
Generated program from GPT:
def append(head: Node, val: int) -> None:
    """Appends a new node with the given value to the 

(True, (3, 1))